# 2 Introduction to machine learning

## 2.1 Data preprocessing

* Getting data from kaggle.com 
* data import
* DataFrame

Download data from https://tinyurl.com/tbjuvka

### Import and explore data

### Fit the data with our custom linear regression model

**Question**: What is the chance level accuracy?

**Exercise**: Try finding the column/threshold that gives the best prediction.

**Exercise**: Name all categorical and numerical columns in the titanic data frame.

**Exercise**: Count the number of rows with missing data.

## 2.2 Logistic regression

* scikit-learn library
* fit/transform methods

### Logistic regression in scikit-learn

In [4]:
from sklearn.linear_model import LogisticRegression

/home/bartosz/.pyenv/versions/3.7.4/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


**Exercise**: Try calculating the precision metrics.

**Exercise**: Investigate the influence of the `C` parameter on the prediction.



### Multiple featues

**Exercise**: Use the `LinearRegression` class from scikit-learn  to fit the `a`, `b` and `c` coefficients of the following function:

`y = a * x1 + b * x2 + c`

where 

```
x1 = [1., 5., 2.]
x2 = [3., 4., 4.]
y  = [4.8, 13.3, 7.1]
```

## 2.3 Cross-validation

* training, validation and tests sets
* validation splits

In [59]:
from sklearn.model_selection import train_test_split

## 2.4 Feature engineering

* one-hot encoding
* value imputation?

In [62]:
from sklearn.preprocessing import OneHotEncoder

**Question**: The code above produces biased results. Can you tell why? 

## 2.5 Putting it all together

* pipelines

In [68]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, ColumnTransformer

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [71]:
df_clean = df[['Survived', 'Age', 'Pclass', 'Fare', 'Embarked', 'Sex', 'SibSp']].dropna()
target = df_clean['Survived']
features = df_clean.drop(columns=['Survived'])

In [72]:
transformer = ColumnTransformer(
  [('categorical', OneHotEncoder(), ['Sex', 'Embarked', 'Pclass']),
   ('numerical', StandardScaler(), ['Age', 'Fare'])], 
  remainder='passthrough')

estimator = make_pipeline(transformer, LogisticRegression())

In [73]:
train_features, validation_features, train_target, validation_target = train_test_split(features, target)

estimator.fit(train_features, train_target).score(validation_features, validation_target)

**Exercise**: Add imputation of missing values to the pipeline. Check out `sklearn.impute`.